In [1]:
import pandas as pd
from pandas import DataFrame,Series
import seaborn as sns
import matplotlib as plt
import scipy
import Advanced_Features
%matplotlib inline

In [2]:
Reg_Season_Rating = pd.read_csv("basketball_csvs/Regular_Season_Team_Rating.csv")
Champions = pd.read_csv("basketball_csvs/Champions.csv")
Playoff_elim = pd.read_csv("basketball_csvs/playoff_elim.csv")
per100_stats = pd.read_csv("basketball_csvs/per_poss_all.csv")
All_stats = pd.read_csv("basketball_csvs/per_game_all.csv")

In [3]:
All_stats.columns

Index(['2P%_game', '2P%_opp_game', '2P.1_opp_game', '2PA_game', '2PA_opp_game',
       '2P_game', '2P_opp_game', '3P%_game', '3P%_opp_game', '3P.1_opp_game',
       '3PA_game', '3PA_opp_game', '3P_game', '3P_opp_game', 'AST_game',
       'AST_opp_game', 'BLK_game', 'BLK_opp_game', 'DRB_game', 'DRB_opp_game',
       'FG%_game', 'FG%_opp_game', 'FGA_game', 'FGA_opp_game', 'FG_game',
       'FG_opp_game', 'FT%_game', 'FT%_opp_game', 'FTA_game', 'FTA_opp_game',
       'FT_game', 'FT_opp_game', 'G_game', 'G_opp_game', 'MP_game',
       'MP_opp_game', 'ORB_game', 'ORB_opp_game', 'PF_game', 'PF_opp_game',
       'PTS_game', 'PTS_opp_game', 'Rk_game', 'Rk_opp_game', 'STL_game',
       'STL_opp_game', 'TOV_game', 'TOV_opp_game', 'TRB_game', 'TRB_opp_game',
       'Team_game', 'Team_opp_game', 'Year'],
      dtype='object')

In [4]:
All_stats['Assist_Ratio'] = All_stats[['AST_game','FGA_game','FTA_game','TOV_game']].apply(Advanced_Features.assist_ratio,axis=1)

In [5]:
All_stats['Pythagorean_Winning_Percentage'] = All_stats[['PTS_game','PTS_opp_game']].apply(Advanced_Features.Pythagorean_Winning_Percentage,axis=1)

In [6]:
All_stats['Efficiency_Differential']=Reg_Season_Rating[['ORtg','DRtg']].apply(Advanced_Features.Efficiency_Differential,axis=1)

In [7]:
All_stats['Defensive_Rebounding_Percentage'] = All_stats[['DRB_game','ORB_opp_game']].apply(Advanced_Features.Defensive_Rebouding_Percentage,axis=1)


In [8]:
All_stats['Possessions_game'] = All_stats[['FGA_game','TOV_game','FTA_game','ORB_game']].apply(Advanced_Features.possessions,axis=1)

In [9]:
All_stats['Possessions_opp_game'] = All_stats[['FGA_opp_game','TOV_opp_game','FTA_opp_game','ORB_opp_game']].apply(Advanced_Features.possessions,axis=1)

In [10]:
All_stats['Offensive_Rebounding_Percentage'] = All_stats[['ORB_game','DRB_opp_game']].apply(Advanced_Features.Offensive_Rebounding_Percentage,axis=1)

In [11]:
All_stats['Pace'] = All_stats[['MP_game','Possessions_game','Possessions_opp_game']].apply(Advanced_Features.pace,axis=1)

In [12]:
All_stats['Play_Percentage'] = All_stats[['FG_game','FGA_game','ORB_game','TOV_game']].apply(Advanced_Features.play_percentage,axis=1)

In [13]:
All_stats['Turnover_Ratio'] = All_stats[['TOV_game','FGA_game','FTA_game','AST_game','TOV_game']].apply(Advanced_Features.Turnover_Ratio,axis=1)

In [39]:
Advanced_features = All_stats[['Year','Team_game','Assist_Ratio',
                              'Pythagorean_Winning_Percentage','Efficiency_Differential',
                              'Defensive_Rebounding_Percentage','Possessions_game','Possessions_opp_game',
                              'Offensive_Rebounding_Percentage','Pace','Play_Percentage','Turnover_Ratio']]

Advanced_features.head()

,Year,Team_game,Assist_Ratio,Pythagorean_Winning_Percentage,Efficiency_Differential,Defensive_Rebounding_Percentage,Possessions_game,Possessions_opp_game,Offensive_Rebounding_Percentage,Pace,Play_Percentage,Turnover_Ratio
0,1986,Atlanta Hawks*,17.270312,0.591152,9.24,0.665909,98.99520,99.13344,0.348624,98.449014,0.474041,12.655573
1,1986,Boston Celtics*,19.801845,0.805120,8.96,0.716418,100.75776,100.62336,0.313107,99.817160,0.487621,11.295898
2,1986,Chicago Bulls*,17.070792,0.362645,7.31,0.673123,99.28320,99.24480,0.351351,98.402974,0.471111,12.193423
3,1986,Cleveland Cavaliers,17.598257,0.395775,2.37,0.684211,100.60416,101.26464,0.302752,100.432239,0.459372,12.011509
4,1986,Dallas Mavericks*,17.838798,0.539461,2.20,0.667411,101.24928,101.28000,0.300000,100.635667,0.485214,10.897632


In [15]:
Unadjusted_Reg_Season_Rating = Reg_Season_Rating.drop(columns = ['Conf','Div','W','L','MOV/A','ORtg/A','DRtg/A','NRtg/A'])
Adjusted_Reg_Season_Rating=Reg_Season_Rating.drop(columns = ['Conf','Div','W','L','MOV','ORtg','DRtg','NRtg'])

In [16]:
Playoff_elim.Year=Playoff_elim.Year.astype(int)
Playoff_elim.head()
Playoff_elim.rename(columns={'Teams':'Team'},inplace=True)

In [17]:
pd.merge(Unadjusted_Reg_Season_Rating,Playoff_elim, on=['Team','Year']).head()

,Rank,Team,W/L%,MOV,ORtg,DRtg,NRtg,Year,Position
0,1,Boston Celtics,0.817,9.41,112.40,103.16,9.24,1986,1
1,2,Milwaukee Bucks,0.695,9.04,112.05,103.09,8.95,1986,3
2,3,Los Angeles Lakers,0.756,7.74,113.69,106.38,7.32,1986,3
3,4,Atlanta Hawks,0.610,2.37,108.43,106.06,2.36,1986,4
4,5,Philadelphia 76ers,0.659,2.35,109.01,106.81,2.21,1986,4


In [ ]:
sns.factorplot(x='ORTG',y = 'W/L%', data = Unadjusted_Reg_Season_Rating,)